负责把vnpy的单个symbol的所有子合约的日线 csv, 合成为复权形式的主力连续合约

不同symbol的主力合约切换是不同的,需要考虑不同的切换逻辑

这里使用通用逻辑: 如果1天volume总和被其他合约超过, 就切换合约

In [5]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import logging
%matplotlib inline

In [6]:
# 单个symbol, vnpy格式的日线数据,
day_bar_data_folder = r"D:\data\vnpy\future\day"
# 生成这个symbol所有子合约的csv path list
day_bar_data_path_list = glob.glob(os.path.join(day_bar_data_folder, "*.csv"))
day_bar_data_path_list = [path for path in day_bar_data_path_list if "#"not in path]

In [7]:
day_bar_data_path_list[0:5]

['D:\\data\\vnpy\\future\\day\\AUDF18.csv',
 'D:\\data\\vnpy\\future\\day\\AUDF19.csv',
 'D:\\data\\vnpy\\future\\day\\AUDF20.csv',
 'D:\\data\\vnpy\\future\\day\\AUDG18.csv',
 'D:\\data\\vnpy\\future\\day\\AUDG19.csv']

In [8]:
symbol_info_frame = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\project_data\my_symbols.csv")

In [9]:
continuous_day_bar_data_folder = r"D:\data\vnpy\future\day_continuous"

In [10]:
#if logger:
    #for handler in logger.handlers[:]:
        #handler.close()
        #logger.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("day_to_day_continuous")
currentDT = datetime.now()
handler = logging.FileHandler("day_to_day_continuous" + currentDT.strftime("%Y_%m_%d_%H_%M_%S") + ".log")
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

for ib_root in symbol_info_frame["ib_root"].to_list():
    continuous_day_data_path = os.path.join(continuous_day_bar_data_folder, ib_root + ".csv")
    # 如果已经合成过了就暂时不要合成了
    if os.path.exists(continuous_day_data_path):
        continue
    
    symbol_paths = [path for path in day_bar_data_path_list if ib_root == os.path.basename(path)[:-7] and ib_root != "MGC"]
    symbol_paths = sorted(symbol_paths)
    symbol_paths = sorted(symbol_paths, key=lambda x: x[-6:-4])
    # 先把所有的数据读进缓存
    data_dict = {}
    

    if len(symbol_paths) > 0:
        for path in symbol_paths:
            frame = pd.read_csv(path)
            frame.loc[:, "date"] = pd.to_datetime(frame['date'], format='%Y-%m-%d')
            frame = frame.sort_values("date")
            #frame.loc[:, "1_days_volume_sum"] = frame.loc[:, "volume"].rolling(min_periods=1, window=1).sum()
            data_dict[path] = frame
        # 先创造一个空的主力连续合约
        continuous_frame = pd.DataFrame()
        # 初始化第一个合约
        init_symbol_path = symbol_paths[0]
        main_frame = data_dict[init_symbol_path]
        current_date = main_frame.iloc[0,: ].loc["date"]

        # 切换合约的累计差值, 初始化为0
        accumulative_adjust = 0
        switch_close_price = 0


        # 已经被切换的合约就不要再切换回来了
        exclusive_list = [init_symbol_path]

        # 初始化一个循环结束的flag
        loop_end = False

        # 初始第一根bar先推给主力合约
        new_row = main_frame[main_frame["date"] == current_date].copy(deep=True)
        # 要把累加值记录下来
        new_row.loc[:, "accumulative_adjust"] = accumulative_adjust
        continuous_frame = pd.concat([continuous_frame, new_row])

        # switch = True

        while(not loop_end):
            # 循环要从大于current date的数据开始, 不包括current date
            filter_main_frame = main_frame[main_frame["date"] > current_date]
            # 如果到最新的日期了后面肯定没数据了, 那就不需要了
            if len(filter_main_frame) == 0:
                loop_end = True
                break
            
            # 每次切换合约的累计差值计算, 第一次不算.
            if switch_close_price:
                accumulative_adjust += filter_main_frame.iloc[0,:].loc["open"] - switch_close_price
            row_count = 0
            for ix, row in filter_main_frame.iterrows():
                row_count += 1
                # 更新当前日子
                current_date = row.loc["date"].to_datetime64()
                # 先不管这么不多, 把当前行的数据推送到主力连续合约
                new_row = row.copy(deep=True)
                # 主力连续合约的价格需要根据复权的累计值调整
                for column in ['open', 'high', 'low', 'close']:
                    new_row.loc[column] = row.loc[column] - accumulative_adjust
                # 要把累加值记录下来
                new_row.loc["accumulative_adjust"] = accumulative_adjust
                continuous_frame = continuous_frame.append(new_row)
                # 更新最后一根k线的close的值, 要原始的价格而不是调整后的
                switch_close_price = row.loc["close"]

                # 接着遍历所有合约的Frame去找有没有当天比当前主力1天volume总和大的合约
                # 如果找到了就隔一天换合约 
                #current_volume_sum = row.loc["1_days_volume_sum"]
                current_volume_sum = row.loc["volume"]
                # 创建一个记录所有超越当前1天volume的symbol dictionary
                switch_symbol_dict = {}
                for path in data_dict.keys():
                    # 已经被切换的合约就不用再管了
                    if path not in exclusive_list:
                        frame = data_dict[path]
                        if current_date in frame["date"].unique():
                            volume_sum = frame[frame["date"] == current_date].loc[:, "volume"].iloc[0]
                            # volume_sum不能为空值
                            if volume_sum:
                                if current_volume_sum:
                                    if volume_sum > current_volume_sum * 0.8 and volume_sum > 10:
                                        # 把这个可能的合约加到dict里面去先
                                        switch_symbol_dict[path] = volume_sum
                # 如果至少找到了一个可切换合约
                if switch_symbol_dict:
                    # 按照1天交易量排序, 字典里volume sum最大的那个合约就切换那个
                    sorted_symbol_list = sorted(switch_symbol_dict.items(), key=lambda x: x[1], reverse = True)
                    new_main_path = sorted_symbol_list[0][0]
                    # 切换合约
                    exclusive_list.append(new_main_path)
                    main_frame = data_dict[new_main_path]
                    logger.info(current_date)
                    logger.info(new_main_path)
                    break
            if row_count == len(filter_main_frame):
                switch_symbol_dict = {}
                for path in data_dict.keys():
                    # 已经被切换的合约就不用再管了
                    if path not in exclusive_list:
                        frame = data_dict[path]
                        if current_date in frame["date"].unique():
                            volume_sum = frame[frame["date"] == current_date].loc[:, "volume"].iloc[0]
                            # volume_sum不能为空值
                            if volume_sum:
                            
                                # 把这个可能的合约加到dict里面去先
                                switch_symbol_dict[path] = volume_sum
                                # 如果至少找到了一个可切换合约
                if switch_symbol_dict:
                    # 按照1天交易量排序, 字典里volume sum最大的那个合约就切换那个
                    sorted_symbol_list = sorted(switch_symbol_dict.items(), key=lambda x: x[1], reverse = True)
                    new_main_path = sorted_symbol_list[0][0]
                    # 切换合约
                    exclusive_list.append(new_main_path)
                    main_frame = data_dict[new_main_path]
                    logger.info(current_date)
                    logger.info(new_main_path)
                else:
                    loop_end = True
        continuous_frame.to_csv(continuous_day_data_path, index=False)

2020-06-28 20:21:20,917 - day_to_day_continuous - INFO - 2008-03-13T00:00:00.000000000
2020-06-28 20:21:20,922 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESM08.csv
2020-06-28 20:21:22,488 - day_to_day_continuous - INFO - 2008-06-12T00:00:00.000000000
2020-06-28 20:21:22,490 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESU08.csv
2020-06-28 20:21:24,037 - day_to_day_continuous - INFO - 2008-09-11T00:00:00.000000000
2020-06-28 20:21:24,039 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESZ08.csv
2020-06-28 20:21:25,595 - day_to_day_continuous - INFO - 2008-12-11T00:00:00.000000000
2020-06-28 20:21:25,597 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESH09.csv
2020-06-28 20:21:27,105 - day_to_day_continuous - INFO - 2009-03-12T00:00:00.000000000
2020-06-28 20:21:27,106 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESM09.csv
2020-06-28 20:21:28,631 - day_to_day_continuous - INFO - 2009-06-11T00:00:00.000000000
2020-06-28 20:21:28,632

2020-06-28 20:22:20,919 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESZ19.csv
2020-06-28 20:22:21,955 - day_to_day_continuous - INFO - 2019-12-13T00:00:00.000000000
2020-06-28 20:22:21,957 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESH20.csv
2020-06-28 20:22:22,973 - day_to_day_continuous - INFO - 2020-03-13T00:00:00.000000000
2020-06-28 20:22:22,975 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESM20.csv
2020-06-28 20:22:23,908 - day_to_day_continuous - INFO - 2020-06-12T00:00:00.000000000
2020-06-28 20:22:23,909 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESU20.csv
2020-06-28 20:22:23,912 - day_to_day_continuous - INFO - 2020-06-12T00:00:00.000000000
2020-06-28 20:22:23,913 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\ESZ20.csv
2020-06-28 20:22:24,460 - day_to_day_continuous - INFO - 2019-06-14T00:00:00.000000000
2020-06-28 20:22:24,462 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\MESU19.csv
2020-06-28 20:22:2

2020-06-28 20:23:02,715 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMZ09.csv
2020-06-28 20:23:03,772 - day_to_day_continuous - INFO - 2009-11-19T00:00:00.000000000
2020-06-28 20:23:03,774 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMF10.csv
2020-06-28 20:23:03,804 - day_to_day_continuous - INFO - 2009-11-19T00:00:00.000000000
2020-06-28 20:23:03,805 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMG10.csv
2020-06-28 20:23:03,844 - day_to_day_continuous - INFO - 2009-11-20T00:00:00.000000000
2020-06-28 20:23:03,846 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMH10.csv
2020-06-28 20:23:06,669 - day_to_day_continuous - INFO - 2010-02-19T00:00:00.000000000
2020-06-28 20:23:06,671 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMJ10.csv
2020-06-28 20:23:06,702 - day_to_day_continuous - INFO - 2010-02-19T00:00:00.000000000
2020-06-28 20:23:06,704 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMK10.csv
2020-06-28 20:23:08

2020-06-28 20:23:34,545 - day_to_day_continuous - INFO - 2013-07-18T00:00:00.000000000
2020-06-28 20:23:34,547 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMU13.csv
2020-06-28 20:23:35,256 - day_to_day_continuous - INFO - 2013-08-16T00:00:00.000000000
2020-06-28 20:23:35,258 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMV13.csv
2020-06-28 20:23:36,063 - day_to_day_continuous - INFO - 2013-09-19T00:00:00.000000000
2020-06-28 20:23:36,065 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMX13.csv
2020-06-28 20:23:36,085 - day_to_day_continuous - INFO - 2013-09-19T00:00:00.000000000
2020-06-28 20:23:36,087 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMF14.csv
2020-06-28 20:23:36,325 - day_to_day_continuous - INFO - 2013-10-01T00:00:00.000000000
2020-06-28 20:23:36,327 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMG14.csv
2020-06-28 20:23:36,842 - day_to_day_continuous - INFO - 2013-10-27T00:00:00.000000000
2020-06-28 20:23:36,844

2020-06-28 20:24:06,239 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMU17.csv
2020-06-28 20:24:06,255 - day_to_day_continuous - INFO - 2017-07-19T00:00:00.000000000
2020-06-28 20:24:06,256 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMV17.csv
2020-06-28 20:24:07,394 - day_to_day_continuous - INFO - 2017-09-19T00:00:00.000000000
2020-06-28 20:24:07,396 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMX17.csv
2020-06-28 20:24:07,409 - day_to_day_continuous - INFO - 2017-09-19T00:00:00.000000000
2020-06-28 20:24:07,411 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMZ17.csv
2020-06-28 20:24:08,459 - day_to_day_continuous - INFO - 2017-11-17T00:00:00.000000000
2020-06-28 20:24:08,461 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMF18.csv
2020-06-28 20:24:08,475 - day_to_day_continuous - INFO - 2017-11-17T00:00:00.000000000
2020-06-28 20:24:08,477 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\QMG18.csv
2020-06-28 20:24:09

...

2020-06-28 20:32:47,534 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPZ14.csv
2020-06-28 20:32:49,142 - day_to_day_continuous - INFO - 2014-12-12T00:00:00.000000000
2020-06-28 20:32:49,143 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPH15.csv
2020-06-28 20:32:50,771 - day_to_day_continuous - INFO - 2015-03-13T00:00:00.000000000
2020-06-28 20:32:50,773 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPM15.csv
2020-06-28 20:32:52,358 - day_to_day_continuous - INFO - 2015-06-12T00:00:00.000000000
2020-06-28 20:32:52,361 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPU15.csv
2020-06-28 20:32:53,914 - day_to_day_continuous - INFO - 2015-09-11T00:00:00.000000000
2020-06-28 20:32:53,915 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPZ15.csv
2020-06-28 20:32:55,459 - day_to_day_continuous - INFO - 2015-12-11T00:00:00.000000000
2020-06-28 20:32:55,461 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GBPH16.csv
2020-06-28 20

2020-06-28 20:34:05,837 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDU13.csv
2020-06-28 20:34:07,583 - day_to_day_continuous - INFO - 2013-09-13T00:00:00.000000000
2020-06-28 20:34:07,585 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDZ13.csv
2020-06-28 20:34:09,297 - day_to_day_continuous - INFO - 2013-12-13T00:00:00.000000000
2020-06-28 20:34:09,300 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDH14.csv
2020-06-28 20:34:10,991 - day_to_day_continuous - INFO - 2014-03-14T00:00:00.000000000
2020-06-28 20:34:10,994 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDM14.csv
2020-06-28 20:34:12,675 - day_to_day_continuous - INFO - 2014-06-13T00:00:00.000000000
2020-06-28 20:34:12,677 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDU14.csv
2020-06-28 20:34:14,357 - day_to_day_continuous - INFO - 2014-09-12T00:00:00.000000000
2020-06-28 20:34:14,358 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\AUDZ14.csv
2020-06-28 20

2020-06-28 20:35:04,552 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADM12.csv
2020-06-28 20:35:05,133 - day_to_day_continuous - INFO - 2011-08-04T00:00:00.000000000
2020-06-28 20:35:05,135 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADU12.csv
2020-06-28 20:35:05,189 - day_to_day_continuous - INFO - 2011-08-09T00:00:00.000000000
2020-06-28 20:35:05,190 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADZ12.csv
2020-06-28 20:35:06,537 - day_to_day_continuous - INFO - 2012-02-10T00:00:00.000000000
2020-06-28 20:35:06,539 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADH13.csv
2020-06-28 20:35:06,651 - day_to_day_continuous - INFO - 2012-02-29T00:00:00.000000000
2020-06-28 20:35:06,653 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADM13.csv
2020-06-28 20:35:07,857 - day_to_day_continuous - INFO - 2012-08-03T00:00:00.000000000
2020-06-28 20:35:07,858 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADZ13.csv
2020-06-28 20

2020-06-28 20:35:26,075 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADG19.csv
2020-06-28 20:35:26,577 - day_to_day_continuous - INFO - 2019-01-28T00:00:00.000000000
2020-06-28 20:35:26,580 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADJ19.csv
2020-06-28 20:35:26,867 - day_to_day_continuous - INFO - 2019-03-01T00:00:00.000000000
2020-06-28 20:35:26,868 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADH20.csv
2020-06-28 20:35:26,919 - day_to_day_continuous - INFO - 2019-03-08T00:00:00.000000000
2020-06-28 20:35:26,921 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADK19.csv
2020-06-28 20:35:27,089 - day_to_day_continuous - INFO - 2019-03-25T00:00:00.000000000
2020-06-28 20:35:27,090 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADM20.csv
2020-06-28 20:35:27,272 - day_to_day_continuous - INFO - 2019-04-23T00:00:00.000000000
2020-06-28 20:35:27,274 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\CADN19.csv
2020-06-28 20

2020-06-28 20:36:55,065 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCZ14.csv
2020-06-28 20:36:58,939 - day_to_day_continuous - INFO - 2014-11-26T00:00:00.000000000
2020-06-28 20:36:58,941 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCG15.csv
2020-06-28 20:37:01,030 - day_to_day_continuous - INFO - 2015-01-28T00:00:00.000000000
2020-06-28 20:37:01,033 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCJ15.csv
2020-06-28 20:37:03,125 - day_to_day_continuous - INFO - 2015-03-29T00:00:00.000000000
2020-06-28 20:37:03,127 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCM15.csv
2020-06-28 20:37:05,175 - day_to_day_continuous - INFO - 2015-05-28T00:00:00.000000000
2020-06-28 20:37:05,176 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCQ15.csv
2020-06-28 20:37:07,344 - day_to_day_continuous - INFO - 2015-07-30T00:00:00.000000000
2020-06-28 20:37:07,347 - day_to_day_continuous - INFO - D:\data\vnpy\future\day\GCZ15.csv
2020-06-28 20:37:11